# Figure 4

NOTE: If you're running this on **Google Colab**, then uncomment and run the next two cells.

In [ ]:
# !git clone https://github.com/Mark-Kramer/Aperiodic-Exponent-Model.git

In [ ]:
# import sys
# sys.path.insert(0,'/content/Aperiodic-Exponent-Model')

---

## Load packages

In [ ]:
import numpy as np
import scipy.io as io
import matplotlib.pyplot as plt
from   matplotlib.backends.backend_pdf import PdfPages

## Load functions for computations and plotting

In [ ]:
from one_over_f_functions import *

## Define the model

In [ ]:
# Simulate the generative model.
def steyn_ross_2003(noise):
    
    tau_e = 0.04;      tau_i = 0.04            # Parameters from [Steyn-Ross et al., PRE, 2003]
    hrest_e = -70.0;   hrest_i = -70.0
    hrev_e = 45.0;     hrev_i = -90.0
    Pee = 1100;        Pei = 1600
    Pie = 1600;        Pii = 1100
    gamma_e = 300;     gamma_i = 65
    G_e = 0.18;        G_i = 0.37
    Nb_ee = 3034.0;    Nb_ei = 3034.0;
    Nb_ie = 536.0;     Nb_ii = 536
    Smax_e = 100;      Smax_i = 100
    theta_e = -60;     theta_i = -60
    g_e = 0.28;        g_i = 0.14
    
    N = 50000+20000;
    dt = 0.0001;
    
    he = np.zeros([N])                         # Output variables
    hi = np.zeros([N])
    t = np.arange(N)*dt                        # Time in [s].
    
    he[0] = hrest_e * np.random.rand()         # Initial conditions
    hi[0] = hrest_i * np.random.rand()
    Iee = np.random.rand();
    Iei = np.random.rand();
    Iie = np.random.rand();
    Iii = np.random.rand();
    Jee = np.random.rand();
    Jei = np.random.rand();
    Jie = np.random.rand();
    Jii = np.random.rand();
    
    for n in np.arange(N-1):                   # Simulate the model
        
        phi_ee = (hrev_e - he[n])/np.abs(hrev_e - hrest_e)
        phi_ei = (hrev_e - hi[n])/np.abs(hrev_e - hrest_i)
        phi_ie = (hrev_i - he[n])/np.abs(hrev_i - hrest_e)
        phi_ii = (hrev_i - hi[n])/np.abs(hrev_i - hrest_i)
        
        Se = Smax_e/(1 + np.exp(-g_e*(he[n]-theta_e)))
        Si = Smax_i/(1 + np.exp(-g_i*(hi[n]-theta_i)))
        
        he[n+1] = he[n]   + dt*1/tau_e * ( (hrest_e - he[n]) + phi_ee*Iee + phi_ie*Iie                 + noise["he"]   *np.random.randn())
        hi[n+1] = hi[n]   + dt*1/tau_i * ( (hrest_i - hi[n]) + phi_ei*Iei + phi_ii*Iii                 + noise["other"]*np.random.randn())
        Iee = Iee + dt*( Jee                                                                           + noise["other"]*np.random.randn())
        Jee = Jee + dt*( -2*gamma_e*Jee - gamma_e*gamma_e*Iee + (Nb_ee*Se + Pee)*G_e*gamma_e*np.exp(1) + noise["other"]*np.random.randn())
        Iei = Iei + dt*( Jei                                                                           + noise["other"]*np.random.randn())
        Jei = Jei + dt*( -2*gamma_e*Jei - gamma_e*gamma_e*Iei + (Nb_ei*Se + Pei)*G_e*gamma_e*np.exp(1) + noise["other"]*np.random.randn())
        Iie = Iie + dt*( Jie                                                                           + noise["other"]*np.random.randn())
        Jie = Jie + dt*( -2*gamma_i*Jie - gamma_i*gamma_i*Iie + (Nb_ie*Si + Pie)*G_i*gamma_i*np.exp(1) + noise["other"]*np.random.randn())
        Iii = Iii + dt*( Jii                                                                           + noise["other"]*np.random.randn())
        Jii = Jii + dt*( -2*gamma_i*Jii - gamma_i*gamma_i*Iii + (Nb_ii*Si + Pii)*G_i*gamma_i*np.exp(1) + noise["other"]*np.random.randn())
                     
    he = he[20000:]
    t  = t[20000:]-t[20000]
    
    return he,t

## Figure 4A-D: Simulate the model, and plot it.

In [ ]:
noise = {"he":[],                           # Set the observable noise,
         "other":50  }                      # ... and the latent noise.
fig, ((ax1, ax2,), (ax3, ax4)) = plt.subplots(2, 2,figsize=(12, 6), dpi=80)

noise["he"]=0;                              # Observable noise = 0.
[he,t] = steyn_ross_2003(noise)             # Simulate model,
[S,f]  = compute_spectrum(he,t)             # ... compute the spectrum & fit aperiodic exponent.
[aperiodic_exponent, x_linear_fit, y_linear_fit] = estimate_aperiodic_exponent(S,f,[50,1000])
make_plots(t,he, f,S, x_linear_fit,y_linear_fit,aperiodic_exponent, ax1,ax3);
ax1.set_ylim([-85, -83]); ax3.set_ylim([-20, 0]); ax3.set_xlim([0,3]); ax1.set(ylabel='[mV]');

noise["he"]=1;                              # Observable noise > 0.
[he,t] = steyn_ross_2003(noise)             # Simulate model,
[S,f]  = compute_spectrum(he,t)             # ... compute the spectrum & fit aperiodic exponent.
[aperiodic_exponent, x_linear_fit, y_linear_fit] = estimate_aperiodic_exponent(S,f,[50,1000])
make_plots(t,he, f,S, x_linear_fit,y_linear_fit,aperiodic_exponent, ax2,ax4);
ax2.set_ylim([-85, -83]); ax4.set_ylim([-20, 0]); ax4.set_xlim([0,3]); ax2.set(ylabel='[mV]')

fig.savefig("./PDFs/Figure-4ABCD.pdf", bbox_inches='tight')

## Figure 4E (Part 1): Iterate over values of noise and plot aperiodic exponent
This part is slow.

In [ ]:
noise = {"he":[],                                          # Prepare the noise variable.
         "other":50  }                                     # ... fix the I-noise,
noise_values = np.arange(0.0,1, 0.01)                      # ... define range of E-noises.
N_replicates = 100                                         # Number of times to repeat simulation.
aperiodic_exponents = np.zeros([np.size(noise_values), N_replicates])

for [count, he_noise] in enumerate(noise_values):          # For each E-noise,
    noise["he"]=he_noise
    print("he_noise: %.4f" % he_noise)
    for j in np.arange(N_replicates):
        [he,t] = steyn_ross_2003(noise)                    # Simulate model N_replicate times,
        [S,f]  = compute_spectrum(he,t)                    # ... compute the spectrum,
        ae = estimate_aperiodic_exponent(S,f,[50,1000])[0]  # ... fit aperiodic exponent,
        aperiodic_exponents[count, j] = ae                 # ... and save it.
                                                           # Save the results.
res = {"noise_values":noise_values, "aperiodic_exponents":aperiodic_exponents}
io.savemat(str('Figure-4E.mat'), res)

## Figure 4E (Part 2): Load the aperiodic exponent, and plot it.

This part is fast.

In [ ]:
# Note: if you're running on Google Colab, then load the .mat file like this:
# res = io.loadmat("/content/Aperiodic-Exponent-Model/Figure-4E.mat");

# Or, if not running on Google Colab, then load the .mat file like this:
res = io.loadmat("Figure-4E.mat")
res = {"noise_values":res["noise_values"], "aperiodic_exponents":res["aperiodic_exponents"],
       "color":"red"};
fig = plt.figure(figsize=(12, 4), dpi=80)
make_plot_ae_vs_noise(res)

fig.savefig("./PDFs/Figure-4E.pdf", bbox_inches='tight')